<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/experiments/experiment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net Experiment 9: Regularization

Adding a dropout layer after each convolutional layer to prevent overfitting.

### 0. Prepare Colab, Define Constants

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#! ls
%cd drive/MyDrive/MachineLearning/
! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls
#! git pull

/content/drive/MyDrive/MachineLearning
fatal: destination path 'Geospatial_ML' already exists and is not an empty directory.
/content/drive/MyDrive/MachineLearning/Geospatial_ML
data_exploration  experiments	     models	   pyproject.toml    scripts
docs		  image_processing   poetry.lock   README.md
evaluation	  metrics_bug.ipynb  prepare_data  requirements.txt


In [3]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    Dropout,
    UpSampling2D
)
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import Sequence
from datetime import datetime
from models.unet_model import unet_2d
from evaluation.evaluation_metrics import EvaluationMetrics
from data_exploration.mask_stats import Mask_Stats
from tensorflow.keras.models import load_model
from models.helpers import save_metrics, predictions_for_models
from evaluation.helpers import plot_metrics, load_metrics_into_df, calc_save_metrics_pred
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.experimental import Adamax

In [4]:
total_tiles = 11121
train_tiles = 6672
test_val_tiles = 2224
data_path = "../data_colab/256_256"
experiment = "experiment_9"
batch_size = 32
tile_size = 256
step_size = 256

### 1. Create Data Generators

In [5]:
train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5), dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256), dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5), dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5), dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.uint8)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()
print()
val_stats = Mask_Stats(val_split_y)
val_stats.print_stats()
print()
test_stats = Mask_Stats(test_split_y)
test_stats.print_stats()

Shape: (6672, 256, 256)
Land pixels: 195058814  44.610 %
Valid pixels: 138904480  31.767 %
Invalid pixels: 103292898  23.623 %
Sum: 6672

Shape: (2224, 256, 256)
Land pixels: 65320265  44.816 %
Valid pixels: 46246663  31.730 %
Invalid pixels: 34185136  23.454 %
Sum: 2224

Shape: (2224, 256, 256)
Land pixels: 64786699  44.450 %
Valid pixels: 46892391  32.173 %
Invalid pixels: 34072974  23.377 %
Sum: 2224


In [6]:
class DataGenerator(Sequence):
    def __init__(self, mmap_x, mmap_y, batch_size):
        self.x_input = mmap_x
        self.y_mask = mmap_y
        self.batch_size = batch_size
        self.num_samples = self.x_input.shape[0]

    # returns number of batches as int
    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    # returns single batch
    def __getitem__(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]

        # normalization
        batch_inputs = batch_inputs/255
        # one-hot-encoding
        batch_masks = np.array([tf.one_hot(item, depth=3).numpy() for item in batch_masks])

        # normalization + one hot encoding
        return batch_inputs, batch_masks

    def getitem_as_img(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]
        # normalization + one hot encoding
        return batch_inputs, batch_masks

In [7]:
# instanciate DataGenerators
batch_size = 32

train_generator = DataGenerator(train_split_x, train_split_y, batch_size)
val_generator = DataGenerator(val_split_x, val_split_y, batch_size)
test_generator = DataGenerator(test_split_x, test_split_y, batch_size)

print(train_generator.__len__())
print(val_generator.__len__())
print(test_generator.__len__())

209
70
70


In [8]:
train_batch = train_generator.__getitem__(9)
val_batch = val_generator.__getitem__(3)
test_batch = test_generator.__getitem__(4)

def print_batch_shapes(batch):
  print(type(batch))
  print(batch[0].shape)
  print(batch[1].shape)
  print()

# check batch shapes
print_batch_shapes(train_batch)
print_batch_shapes(val_batch)
print_batch_shapes(test_batch)

# check normalization
print('Check normalization')
print(train_batch[1].max())
print(train_batch[1].min())

print(val_batch[1].max())
print(val_batch[1].min())

print(test_batch[1].max())
print(test_batch[1].min())

print()
# check one-hot-encoding
print('check one hot encoding')
print(train_batch[0].max())
print(train_batch[0].min())

print(val_batch[0].max())
print(val_batch[0].min())

print(test_batch[0].max())
print(test_batch[0].min())

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

Check normalization
1.0
0.0
1.0
0.0
1.0
0.0

check one hot encoding
1.0
0.0
1.0
0.0
1.0
0.0


### 3. Model training
execute with premium GPU, High RAM

In [9]:
!nvidia-smi

Wed Jun 14 23:07:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    48W / 400W |    709MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
def unet_2d(input_shape, num_classes, dropout=None):

    # Define the input layer
    inputs = Input(input_shape)

    # Downsample layers
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    if dropout:
      conv1 = Dropout(dropout)(conv1)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    if dropout:
      conv2 = Dropout(dropout)(conv2)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    if dropout:
      conv3 = Dropout(dropout)(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)

    # Upsample layers
    up4 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)
    if dropout:
      conv4 = Dropout(dropout)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)

    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
    if dropout:
      conv5 = Dropout(dropout)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

    # Define the model
    model = Model(inputs=[inputs], outputs=[output])

    return model

testing 3 different dropout rates, 3 runs with each rate => in total 9 runs

In [ ]:
all_metrics = []

dropout_rates = [0.1, 0.2] #None
count = 0

for dropout in dropout_rates:
  for i in range(1,3):

    model_metrics = []
    model_name = f'{tile_size}_{step_size}_run_{count}_dropout_{dropout}'
    model_path = f'../models/{experiment}/model_{model_name}.h5'

    print(f'{count}. dropout: {dropout} Started at: {datetime.now()}')

    # Define the mIoU metric
    mean_iou = tf.keras.metrics.OneHotMeanIoU(num_classes=3, name='mean_iou')
    invalid_iou = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[0], name='invalid_iou')
    valid_iou = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[1], name='valid_iou')
    land_iou = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[2], name='land_iou')

    # compiling model
    model = unet_2d((256, 256, 5), 3, dropout)
    model.compile(optimizer=Adamax(), loss=categorical_crossentropy, metrics=[mean_iou, invalid_iou, valid_iou, land_iou, 'accuracy'])
    print(model.summary())

    # callbacks
    early_stop_loss = EarlyStopping(monitor='val_loss', mode='min', patience=15)
    early_stop_acc = EarlyStopping(monitor='val_mean_iou', mode='max', patience=15)
    checkpoint = ModelCheckpoint(model_path, monitor="val_mean_iou", mode="max", save_best_only=True, verbose=1)

    # training
    model_history = model.fit(x=train_generator, epochs=100, validation_data=val_generator,
                              callbacks=[early_stop_loss, early_stop_acc, checkpoint])


    # Save model history
    with open(f'../models/{experiment}/history_{model_name}.pkl', 'wb') as file_pi:
        pickle.dump(model_history.history, file_pi)
    print('saving history completed')

    print(f'{count} Finished at: {datetime.now()}')
    print(f'{count} Metrics')
    print(model.metrics_names)
    print('training metrics')
    train_metrics = model.evaluate(train_generator, verbose = 2)
    print(train_metrics)
    print('validation metrics')
    val_metrics = model.evaluate(val_generator, verbose = 2)
    print(val_metrics)
    print('test metrics')
    test_metrics = model.evaluate(test_generator, verbose = 2)
    print(test_metrics)
    all_metrics.append(train_metrics)
    all_metrics.append(val_metrics)
    all_metrics.append(test_metrics)

    with open(f'../metrics/{experiment}_{count}.pkl', 'wb') as file_pi:
        pickle.dump(all_metrics, file_pi)
    print('saving metrics completed')
    count += 1

0. dropout: 0.1 Started at: 2023-06-14 23:07:47.911185
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  2944        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 256, 256, 64  0           ['conv2d[0][0]']                 
                                )      